In [ ]:
import numpy as np
import sympy as sp

from sympy.solvers.inequalities import reduce_inequalities

In [ ]:
def reduce_ineq(ineq: sp.core.relational._Inequality, var: sp.core.symbol.Symbol):
    interval = reduce_inequalities(ineq, var)
    args = interval.args if isinstance(interval, sp.And) else [interval]
    for inter in args:
        inter = inter.canonical
        if not (inter.has(sp.oo) or inter.has(-sp.oo)):
            return inter

In [ ]:
def fourier_motzkin_eliminate_var(
    inequalites: tuple[sp.core.relational._Inequality],
    var: sp.core.symbol.Symbol,
    all_vars: tuple[sp.core.symbol.Symbol]
) -> tuple[sp.core.relational._Inequality]:
    less_exprs = []
    great_exprs = []
    zero_ineqs = []
    for ineq in inequalites:
        if var in ineq.free_symbols:
            ineq = reduce_ineq(ineq, var)
            if isinstance(ineq, sp.core.relational.GreaterThan):
                great_exprs.append(ineq.rhs)
            else:
                less_exprs.append(ineq.rhs)
        else:
            zero_ineqs.append(ineq)
    new_ineqs = []
    for ge in great_exprs:
        for le in less_exprs:
            new_in = ge <= le
            diff = ge - le
            if any(diff.has(v) for v in all_vars):
                new_ineqs.append(ge <= le)
    return tuple(zero_ineqs + new_ineqs)

In [ ]:
l, m, n = sp.symbols("l m n", integers=True)
i, j, k = sp.symbols("i j k", integers=True)

In [ ]:
ineqs = (
    0 <= j,
    j <= n-1,
    1 <= i - 2*j - k,
    i - 2*j - k <= m - 2,
    1 <= k,
    k <= l-2
)

In [ ]:
fourier_motzkin_eliminate_var(ineqs, k, (i,j,k))

In [ ]:
def simplify_bounds(ineqs, var):
    lower_bounds = []
    upper_bounds = []
    for ineq in ineqs:
        if var in ineq.free_symbols:
            ineq = reduce_ineq(ineq, var)
            if ineq.has(var):
                bound = sp.together(ineq.rhs)
                if isinstance(ineq, sp.core.relational.GreaterThan):
                    lower_bounds.append(bound)
                else:
                    upper_bounds.append(bound)
    return (lower_bounds, upper_bounds)

In [ ]:
simplify_bounds(ineqs, k)

In [ ]:
def extract_bounds(ineqs, all_vars, last_index):
    dim = len(all_vars)
    bounds = {}
    curr_index = dim - 1

    while curr_index > last_index:
        curr_var = all_vars[curr_index]
        bounds[curr_var] = simplify_bounds(ineqs, curr_var)
        ineqs = fourier_motzkin_eliminate_var(ineqs, curr_var, all_vars)
        curr_index -= 1

    last_var = all_vars[last_index]
    bounds[last_var] = simplify_bounds(ineqs, last_var)
    return bounds

In [ ]:
extract_bounds(ineqs, (i,j,k), 1)

In [ ]:
def generate_bounds(ineqs, all_vars, last_target_index=0):
    vars_bounds = extract_bounds(ineqs, all_vars, last_target_index)
    final_bounds = {}
    for var, bounds in vars_bounds.items():
        lowers, uppers = bounds
        ceiled_lowers = map(lambda l: sp.ceiling(l)
                            if isinstance(l, sp.core.mul.Mul) and any(map(lambda l: l.is_rational and not l.is_integer, l.as_two_terms())) else l, lowers)
        floored_uppers = map(lambda l: sp.floor(l)
                             if isinstance(l, sp.core.mul.Mul) and any(map(lambda l: l.is_rational and not l.is_integer, l.as_two_terms())) else l, uppers)
        final_bounds[var] = (
            sp.Max(*ceiled_lowers),
            sp.Min(*floored_uppers)
        )
    return final_bounds

In [ ]:
bounds = generate_bounds(ineqs, (i,j,k))
bounds

In [ ]:
"int jstart = " + sp.printing.ccode(bounds[j][0], standard="C99")

In [ ]:
"int jend = " + sp.printing.ccode(bounds[j][1], standard="C99")

In [ ]:
def reindex(T_inv: np.array, x: sp.Matrix, ls: sp.Matrix, us: sp.Matrix):
    system = T_inv * x
    ineqs = []
    for i, t in enumerate(system):
        ineqs.append(t >= ls[i])
        ineqs.append(t <= us[i])
    all_vars = tuple(xx for xx in x)
    return generate_bounds(ineqs, all_vars)

In [ ]:
def invert_integer_matrix(mat: np.ndarray):
    return np.linalg.inv(mat).round().astype(np.int)

In [ ]:
# Example 1
reindex(invert_integer_matrix([
    [1, 1],
    [0, 1]
]), sp.Matrix([[i], [j]]), sp.Matrix([[1], [1]]), sp.Matrix([[n], [m]]))

In [ ]:
# Example 1a
reindex(invert_integer_matrix([
    [1, 1],
    [1, 0]
]), sp.Matrix([[i], [j]]), sp.Matrix([[1], [1]]), sp.Matrix([[n], [m]]))

In [ ]:
# Example 2
reindex(invert_integer_matrix([
    [2, 1],
    [1, 0]
]).astype(np.int), sp.Matrix([[i], [j]]), sp.Matrix([[1], [1]]), sp.Matrix([[n], [m]]))

In [ ]:
# Example 3
reindex(invert_integer_matrix([
    [3, 2],
    [1, 1]
]).round().astype(np.int), sp.Matrix([[i], [j]]), sp.Matrix([[1], [1]]), sp.Matrix([[n], [m]]))

In [ ]:
# Example 4
reindex(invert_integer_matrix([
    [1, 1, 1],
    [0, 1, 0],
    [0, 0, 1]
]).round().astype(np.int), sp.Matrix([[i], [j], [k]]), sp.Matrix([[1], [1], [1]]), sp.Matrix([[n], [m], [l]]))

In [ ]:
# Example 5
reindex(invert_integer_matrix([
    [2, 1, 1],
    [1, 0, 0],
    [0, 0, 1]
]).round().astype(np.int), sp.Matrix([[i], [j], [k]]), sp.Matrix([[1], [1], [1]]), sp.Matrix([[n], [m], [l]]))

In [ ]:
# Example 6
reindex(invert_integer_matrix([
    [2, 1, 1],
    [1, 1, 0],
    [0, 0, 1]
]).round().astype(np.int), sp.Matrix([[i], [j], [k]]), sp.Matrix([[1], [1], [1]]), sp.Matrix([[n], [m], [l]]))

In [ ]:
T = invert_integer_matrix([
    [2, 1, 1],
    [1, 1, 0],
    [0, 0, 1]
]).round().astype(np.int)
idxs = sp.Matrix([[i], [j], [k]])
ls = sp.Matrix([[1], [1], [1]])
us = sp.Matrix([[n], [m], [l]])

In [ ]:
%timeit reindex(T, idxs, ls, us)